# Introdução à Recuperação de Informações

# Lista de exercícios 1

## Ana Carolina Wagner G. de Barros

___

#### 1. Carregando as bibliotecas e preparando o ambiente

Vamos utilizar a biblioteca NLTK para processamento de linguagem natural. Em seguida vamos importar mais coisas necessárias para o nosso trabalho. Note que estamos baixando a obra completa de Machado de Assis, com a qual iremos alimentar nosso índice. Vamos também baixar o banco de stopwords do NLTK. Stop words são um conjunto de palavras que normalmente carregam baixo conteúdo semântico e portanto não são alvo de buscas.

In [1]:
import numpy as np
import pandas as pd
import os

import nltk
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer

import string

from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nltk.download('stopwords')
nltk.download('machado')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/acwgdb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package machado to /Users/acwgdb/nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

Lendo o texto puro dos livros de Machado.

In [3]:
textos = []
for i in machado.fileids():
    textos.append(machado.raw(i))

Carregando a lista de stopwords em lingua portuguesa para limpeza dos textos. 

In [4]:
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

#### 2. Preparando o corpus com e sem *stemming*

A técnica de *stemming* consiste em reduzir o termo ao seu radical, removendo afixos e vogais temáticas. Por exemplo, a palavra “frequentemente” após esse processo se torna “frequent”, a palavra “copiar” após esse processo se torna “copi”.

In [5]:
# Sem stemming
textos_normal = []
for texto in textos:
    tlimpo = [token.lower() for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_normal.append(tlimpo)

In [6]:
# Com stemming
textos_stem = []
for texto in textos_normal:
    tlimpo = [stemmer.stem(token) for token in texto]
    textos_stem.append(tlimpo)

In [7]:
print(textos_normal[0][0:10])
print(textos_stem[0][0:10])

['conto', 'contos', 'fluminenses', '1870', 'contos', 'fluminenses', 'texto', 'fonte', 'obra', 'completa']
['cont', 'cont', 'fluminens', '1870', 'cont', 'fluminens', 'text', 'font', 'obra', 'complet']


#### 3. Criando os índices invertidos

In [8]:
indice_normal = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_normal):
    for term in t:
        indice_normal[term].add(tid)

In [9]:
indice_stem = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_stem):
    for term in t:
        indice_stem[term].add(tid)

#### 4. Função de busca

O operador segue funcional até outro operador ser sugerido. O operador padrão é o 'or'. 

Operadores disponíveis: 'and', 'or' e 'not'.

Eles são executados nesta mesma sequência.

In [207]:
def busca(consulta, indice):
    
    tokens = WordPunctTokenizer().tokenize(consulta)

    resultado = set()
    operador = 'or'
    
    for token in tokens:
        if token == 'and':
            operador = 'and'
            
        elif token == 'or':
            operador = 'or'
        
        elif token == 'not':
            operador = 'not'
            
        else:
            resultado_temp = indice[token]
            if operador == 'and':
                resultado = resultado & resultado_temp
            
            elif operador == 'not':
                resultado = resultado - resultado_temp
                
            else:
                resultado = resultado | resultado_temp
    
    return resultado

### Exercício 1: Truncagem e revocação.

Baseando-se no indice invertido construído na prática 1, calcule a diferença de revocação com e sem a utilização de "stemming", ou truncagem na construção do índice.

**Precisão:** "Quantos elementos selecionados são relevantes?"

$$Pre = \frac{|\{ \text{documentos recuperados} \cap \text{documentos relevantes} \}|}{|\{ \text{documentos recuperados}\}|}$$

A precisão corresponde ao número de resultados corretos dividido pelo número de todos os resultados retornados.

**Revocação:** "Quantos elementos relevantes foram selecionados?"

$$Rev = \frac{|\{ \text{documentos recuperados} \cap \text{documentos relevantes} \}|}{|\{ \text{documentos relevantes}\}|}$$

A revocação corresponde ao número de resultados corretos dividido pelo número de resultados que deveriam ter sido apresentados.


<img src="https://upload.wikimedia.org/wikipedia/commons/a/ab/Precisão_e_revocação.png" width="200">

In [11]:
def PreRev(recuperados, relevantes):
    
    # precisao 
    pre = len(recuperados &  relevantes)/len(recuperados)
    # revocacao
    rev = len(recuperados &  relevantes)/len(relevantes)
    
    return pre,rev

In [205]:
# Palavra a ser consultada
palavra = 'contos'

In [208]:
# Resultados da busca
res_normal = busca(palavra,indice_normal)
res_stem = busca(stemmer.stem(palavra), indice_stem)

print('Quantidade de Resultados Stemmizado: ' + str(len(res_stem)))

print('Quantidade de Resultados Não Stemmizado: ' + str(len(res_normal)))


Quantidade de Resultados Stemmizado: 219
Quantidade de Resultados Não Stemmizado: 74


A revocação é calculada como a proporção entre o resultado de stemming com o resultado sem stemming.

$$Rev = \frac{Res_\text{normal}}{Res_\text{stem}}$$

In [209]:
print('Precisão: ' +  str(PreRev(res_normal, res_stem)[0]))
print('Revocação: ' + str(PreRev(res_normal, res_stem)[1]))

Precisão: 1.0
Revocação: 0.3378995433789954


### Exercício 2: Expansão de consultas
Crie grupos de equivalência para alguns termos de busca e calcule a diferença em termos de revocação e, possivelmente precisão, na resposta a consultas expandidas e não expandidas. Dica: use tempos verbais, pluralização, sinônimos, etc.

In [15]:
termos = ['contos', 'conto', 'contar', 'contando', 'narração', 'narrações'] 
#termos = ['ator', 'atores','atriz','atrizes']

In [210]:
res_normal2 = set()

print('Documentos encontrados sem Stemming:')
for termo in termos:
    res = busca(termo, indice_normal)
    res_normal2.update(res)
    print(' {}: {}'.format(termo, len(res)))
print('\n')
print('TOTAL: {}'.format(len(res_normal2)))

Documentos encontrados sem Stemming:
 contos: 74
 conto: 166
 contar: 128
 contando: 58
 narração: 84
 narrações: 13


TOTAL: 201


In [17]:
res_stem2 = set()

print('Documentos encontrados sem Stemming:')
for termo in termos:
    res = busca(stemmer.stem(termo), indice_stem)
    res_stem2.update(res)
    print(' {}: {}'.format(termo, len(res)))
print('\n')
print('TOTAL: {}'.format(len(res_stem2)))

Documentos encontrados sem Stemming:
 contos: 219
 conto: 219
 contar: 219
 contando: 219
 narração: 84
 narrações: 13


TOTAL: 221


In [18]:
print('Precisão: ' +  str(PreRev(res_normal2, res_stem2)[0]))
print('Revocação: ' + str(PreRev(res_normal2, res_stem2)[1]))

Precisão: 1.0
Revocação: 0.9095022624434389


- Houve uma grande melhora na revocação.

###  Exercício 3: Verificação ortográfica
Implemente uma expansão de consulta por meio da correção ortográfica. Utilize o corretor ortográfico Pyenchant para fazer as correções.

Tutorial: https://github.com/rfk/pyenchant/blob/master/website/content/tutorial.rst

In [19]:
import enchant

In [26]:
# dicionario
d = enchant.Dict("pt_BR")

Vamos implementar uma função que checa as palavras de busca e sugere novas palavras se algo estiver errado. 

In [41]:
def corrige_palavra(palavra):
    
    if not d.check(palavra):
        
        print('Você quis dizer: ')
        for p in d.suggest(palavra)[:5]:
            print('-', p)

In [59]:
corrige_palavra('ontos')

Você quis dizer: 
- om tos
- otos
- onos
- tontos
- contos


In [86]:
def busca_correta(palavra,ind,dic):

    # faz a verificação ortográfica e a busca sobre os termos possíveis
    
    if not dic.check(palavra):
        resultado = set()
        for token in dic.suggest(palavra):
            resultado = resultado | ind[token]
    else:
        resultado = ind[termo]
        
    return resultado

def busca2(consulta, indice):
    
    d = enchant.Dict("pt_BR")
    tokens = WordPunctTokenizer().tokenize(consulta)

    resultado = set()
    operador = 'or'
    
    for token in tokens:
        if token == 'and':
            operador = 'and'
            
        elif token == 'or':
            operador = 'or'
        
        elif token == 'not':
            operador = 'not'
            
        else:
            
            #resultado_temp = indice[token]
            # Caso um termo esteja escrito errado, faz-se a busca sobre todas as correções sugeridas
            resultado_temp = busca_correta(token, indice, d)
            
            if operador == 'and':
                resultado = resultado & resultado_temp
            
            elif operador == 'not':
                resultado = resultado - resultado_temp
                
            else:
                resultado = resultado | resultado_temp
    
    return resultado

In [87]:
# Palavra a ser consultada
palavra = 'ontos'

In [88]:
# Resultados da busca usando a função criada para os exercícios anteriores (busca)
res_normal = busca(palavra,indice_normal)
res_stem = busca(stemmer.stem(palavra), indice_stem)

print('Quantidade de Resultados Stemmizado: ' + str(len(res_stem)))

print('Quantidade de Resultados Não Stemmizado: ' + str(len(res_normal)))

Quantidade de Resultados Stemmizado: 0
Quantidade de Resultados Não Stemmizado: 0


In [89]:
# Resultados da busca usando a função adaptada que usa a correção ortográfica (busca2)
res_normal = busca2(palavra,indice_normal)
res_stem = busca2(stemmer.stem(palavra), indice_stem)

print('Quantidade de Resultados Stemmizado: ' + str(len(res_stem)))

print('Quantidade de Resultados Não Stemmizado: ' + str(len(res_normal)))

Quantidade de Resultados Stemmizado: 0
Quantidade de Resultados Não Stemmizado: 93


###  Exercício 4: Consultas por frases
Implemente um indice invertido que permita consulta por frases, conforme definido na aula 2.

O objetivo é buscar uma frase, exata ou não, dentro de uma coleção de documentos. 

#### 1. Criando novo indice para consulta por frases

In [165]:
textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)
    
indice_frase = defaultdict(lambda: defaultdict(list))

for texto_id, texto in enumerate(textos_limpos):
    for token_id, token in enumerate(texto):
        indice_frase[token][texto_id].append(token_id)

#### 2. Testando o índice

In [166]:
indice_frase[stemmer.stem("Capitu")].keys()

dict_keys([230])

In [167]:
len(indice_frase[stemmer.stem("Capitu")][230])

341

Capitu (Capitolina) é uma personagem da obra **Dom Casmurro** de Machado de Assis. Assim, podemos dizer que o documento 230 corresponde a referida obra. 

#### 3. Redefinindo a nova função de busca

Auxliar da função **busca_frase**: compara a proximidade entre termos de duas listas e devolve a média dos valores quando proximos. Utilziamos a média, pois ela capta os termos que estão próximos, pelo menos boa parte do conjunto de tokens próximos. 

In [318]:
def proximidade_termos(lista1, lista2, distancia):
    resultado = []
    
    for i in lista1:
        for j in lista2:
            if abs(i-j) <= distancia:
                resultado.append(round((i+j)/2))
                
    return resultado

In [359]:
def busca_frase(frase, indice, distancia=1):
    
    tokens = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(frase) if token not in swu]
    
    # Verificar quais documentos possuem todos os termos
    matches = indice[tokens[0]].keys()
    
    for token in tokens[1:]:
        matches = matches & indice[token].keys()    
        
    # Comparar as posições dentro dos documentos em comum
    resultado = defaultdict(list)
    
    for documento in matches:
        
        resultado_temp = indice[tokens[0]][documento]
        
        for token in tokens[1:]:
            resultado_temp = proximidade_termos(resultado_temp,indice[token][documento],distancia)
            
        if resultado_temp:
            resultado[documento] = resultado_temp
            
    return resultado

In [362]:
busca_frase("olhos de ressaca", indice_frase)

defaultdict(list, {230: [10018, 10314, 13638, 13700, 13742, 36264, 39560]})

De fato, "olhos de ressaca" e como Machado de Assis em Dom Casmurro, define os olhos de Capitu.

###  Exercício 5: Consulta híbrida.
Modifique a solução acima para permitir respostas alternativas caso a frase não retorne resultados. Por exemplo, retornar, documentos que contenham parte da frase, ou uma busca booleana simples combinando as palavras da frase.